https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

https://cookbook.openai.com/examples/how_to_call_functions_for_knowledge_retrieval

In [1]:
import markdown
from IPython.display import display, Markdown
from agents import Agent, Runner, trace
import asyncio
from openai.types.responses import ResponseTextDeltaEvent

from dotenv import load_dotenv
import os
load_dotenv()
    
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
    
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
# disable tracing

OPENAI_AGENTS_DISABLE_TRACING=1
from agents import Agent, Runner, set_tracing_disabled
# Explicitly disable tracing
set_tracing_disabled(disabled=True)

import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-4o"
client = OpenAI()

OpenAI API Key set: Yes
Google API Key set: Yes


In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [4]:
from geopy.geocoders import Nominatim

def get_lat_long(location_name):
    """
    Converts a location name to latitude and longitude coordinates.

    Args:
        location_name (str): The name of the location.

    Returns:
        tuple: A tuple containing the latitude and longitude, 
               or None if the location is not found.
    """
    geolocator = Nominatim(user_agent="geocoding_app")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

In [5]:
from typing import Optional

def get_current_weather(location: str, Farenheit: Optional[bool] = False) -> str:
    """
    Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        Farenheit: the temperature
    """
    #return "The weather is fine and temperatures below 50°F"
    coordinates = get_lat_long(location)
    latitude, longitude = coordinates
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&temperature_unit=fahrenheit"
    )
    data = response.json()
    return data["current"]


In [6]:
from datetime import datetime

def get_todays_date() -> str:
    """Get today's date in YYYY-MM-DD format."""
    date = datetime.now().strftime("%Y-%m-%d")
    return date
get_todays_date()

'2025-05-28'

In [7]:
import requests
from openai import OpenAI
from pydantic import BaseModel, Field
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&temperature_unit=fahrenheit"
    )
    data = response.json()
    return data["current"]


In [8]:
latitude=34.0522
longitude=-118.2437
get_weather(latitude, longitude)

{'time': '2025-05-28T21:45',
 'interval': 900,
 'temperature_2m': 68.1,
 'relative_humidity_2m': 71,
 'precipitation': 0.0,
 'wind_speed_10m': 17.0}

In [9]:
get_current_weather("Los Angeles")

{'time': '2025-05-28T21:45',
 'interval': 900,
 'temperature_2m': 68.1,
 'relative_humidity_2m': 71,
 'precipitation': 0.0,
 'wind_speed_10m': 17.0}

In [10]:
"""
docs: https://platform.openai.com/docs/guides/function-calling
"""
#'''
from openai import OpenAI

client = OpenAI()

weather_tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=weather_tools
)

print(completion.choices[0].message.tool_calls)
#'''

[ChatCompletionMessageToolCall(id='call_HGVUeoBDXWqk66teEvaBLpn9', function=Function(arguments='{"location":"Paris, France"}', name='get_current_weather'), type='function')]


In [11]:
system_prompt = "You are a helpful weather assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Los Angeles today?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=weather_tools,
)

In [12]:
completion.model_dump()

{'id': 'chatcmpl-BcJ0y3RpC1i2kDYxR6YsJwMqftcV3',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_1mnilnHdxEwGaqSqPpOcv5Tx',
      'function': {'arguments': '{"location":"Los Angeles, USA"}',
       'name': 'get_current_weather'},
      'type': 'function'}]}}],
 'created': 1748469256,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_a288987b44',
 'usage': {'completion_tokens': 18,
  'prompt_tokens': 73,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [13]:
def call_function(name, args):
    if name == "get_current_weather":
        return get_current_weather(**args)


for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)
    print(f"Name = {name} Args = {args}")
    result = call_function(name, args)
    print(result)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

Name = get_current_weather Args = {'location': 'Los Angeles, USA'}
{'time': '2025-05-28T21:45', 'interval': 900, 'temperature_2m': 68.1, 'relative_humidity_2m': 71, 'precipitation': 0.0, 'wind_speed_10m': 17.0}


In [14]:
date_tools = [
    {

        "type": "function",
        "function": {
            "name": "get_todays_date",
            "description": "Get todays date",
            "parameters": {
                "type": "object",
                "properties": {
                },
            },
        }
    },
]

In [15]:
from typing import Optional


def get_current_weather_with_units(location: str, format: str) -> str:
#def get_current_weather(location: str, Farenheit: Optional[bool] = False) -> str:
    """
    Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        Farenheit: the temperature
    """
    # if format not in ["celsius", "fahrenheit"]:
    #     return "Error: format must be 'celsius' or 'fahrenheit'."
    
    # if format == "fahrenheit":
    #     return f"Getting weather for {location} in Fahrenheit."
    # else:
    #     return f"Getting weather for {location} in Celsius."


    #return "The weather is fine and temperatures below 50°F"
    coordinates = get_lat_long(location)
    latitude, longitude = coordinates
    # response = requests.get(
    #     f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&temperature_unit=fahrenheit"
    # )
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
        f"&temperature_unit={format}"
    )
    data = response.json()
    return data["current"]


In [16]:
print(get_current_weather_with_units("San Francisco, CA", "celsius"))
print(get_current_weather_with_units("New York, NY", "fahrenheit"))


{'time': '2025-05-28T21:45', 'interval': 900, 'temperature_2m': 14.5, 'relative_humidity_2m': 84, 'precipitation': 0.0, 'wind_speed_10m': 29.5}
{'time': '2025-05-28T21:45', 'interval': 900, 'temperature_2m': 57.3, 'relative_humidity_2m': 90, 'precipitation': 0.5, 'wind_speed_10m': 10.4}


In [17]:
import requests

def get_n_day_weather_forecast(location: str, format: str, num_days: int) -> str:
    # Validate format
    # if format not in ["celsius", "fahrenheit"]:
    #     return "Error: format must be 'celsius' or 'fahrenheit'."

    # Example: Resolve coordinates from the location (you'd use geocoding API for real use)

    #latitude, longitude = 37.7749, -122.4194  # Default to San Francisco, CA
    coordinates = get_lat_long(location)
    latitude, longitude = coordinates

    
    # Make API request
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        f"&temperature_unit={format}"
        f"&forecast_days={num_days}"
    )
    
    # Check if the response is successful
    if response.status_code != 200:
        return f"Error fetching weather data: {response.status_code}"

    # Parse the response JSON
    data = response.json()    
    
    # Extract forecast data
    forecast_data = data.get("daily", {}).get("temperature_2m_max", [])

    # if not forecast_data:
    #     return "Error: Unable to retrieve forecast data."
    
    # Prepare the result string
    forecast_str = f"{num_days}-day weather forecast for {location} in {format.capitalize()}:\n"
    
    for day_idx in range(num_days):
        max_temp = forecast_data[day_idx]
        min_temp = data["daily"]["temperature_2m_min"][day_idx]
        forecast_str += f"Day {day_idx + 1}: Max Temp: {max_temp}° {format.capitalize()}, Min Temp: {min_temp}° {format.capitalize()}\n"
    #print(f" Forcast = {forecast_str}")
    return forecast_str


In [18]:
print(get_n_day_weather_forecast("San Francisco, CA", "celsius", 3))
print(get_n_day_weather_forecast("New York, NY", "fahrenheit", 5))


3-day weather forecast for San Francisco, CA in Celsius:
Day 1: Max Temp: 16.3° Celsius, Min Temp: 10.9° Celsius
Day 2: Max Temp: 19.8° Celsius, Min Temp: 10.6° Celsius
Day 3: Max Temp: 23.5° Celsius, Min Temp: 9.7° Celsius

5-day weather forecast for New York, NY in Fahrenheit:
Day 1: Max Temp: 67.1° Fahrenheit, Min Temp: 56.7° Fahrenheit
Day 2: Max Temp: 68.1° Fahrenheit, Min Temp: 56.4° Fahrenheit
Day 3: Max Temp: 73.1° Fahrenheit, Min Temp: 60.8° Fahrenheit
Day 4: Max Temp: 70.7° Fahrenheit, Min Temp: 58.3° Fahrenheit
Day 5: Max Temp: 65.7° Fahrenheit, Min Temp: 51.5° Fahrenheit



https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [19]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather_with_units",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [20]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Please provide the location for which you would like to know the weather today.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [21]:
messages.append({"role": "user", "content": "I'm in Los Angeles, CA."})
chat_response = chat_completion_request(
    messages, tools=tools
)
print(chat_response)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletion(id='chatcmpl-BcJ2Oj2bRTTnBBcg1dP6Fc8UdJwlF', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DrrECXR9NtTk4LlDnb28yuLH', function=Function(arguments='{"location":"Los Angeles, CA","format":"fahrenheit"}', name='get_current_weather_with_units'), type='function')]))], created=1748469344, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_9bddfca6e2', usage=CompletionUsage(completion_tokens=25, prompt_tokens=214, total_tokens=239, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DrrECXR9NtTk4LlDnb28yuLH', function=Function(arguments='{"location":"Los Angeles, CA","format":"fahrenheit"}', name='get_current_weather_with_units'), type='function')])

In [22]:
# messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
# chat_response = chat_completion_request(
#     messages, tools=tools
# )
# print(chat_response)
# assistant_message = chat_response.choices[0].message
# messages.append(assistant_message)
# assistant_message


In [23]:
def call_function(name, args):
    if name == "get_current_weather_with_units":
        return get_current_weather_with_units(**args)


for tool_call in chat_response.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)
    print(f"Name = {name} Args = {args}")
    result = call_function(name, args)
    print(f"\nResult = {result}")
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

Name = get_current_weather_with_units Args = {'location': 'Los Angeles, CA', 'format': 'fahrenheit'}

Result = {'time': '2025-05-28T21:45', 'interval': 900, 'temperature_2m': 68.1, 'relative_humidity_2m': 71, 'precipitation': 0.0, 'wind_speed_10m': 17.0}


In [24]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Los Angeles, CA over the next 3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_onC9AGCviZsfL9TIxtgHkfOF', function=Function(arguments='{"location":"Los Angeles, CA","format":"fahrenheit","num_days":3}', name='get_n_day_weather_forecast'), type='function')])

In [25]:
def call_function(name, args):
    if name == "get_n_day_weather_forecast":
        print(f"\n calling function = {name}")
        return get_n_day_weather_forecast(**args)


for tool_call in chat_response.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)
    print(f"Name = {name} Args = {args}")
    result = call_function(name, args)
    print(f"\nResult = {result}")
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

Name = get_n_day_weather_forecast Args = {'location': 'Los Angeles, CA', 'format': 'fahrenheit', 'num_days': 3}

 calling function = get_n_day_weather_forecast

Result = 3-day weather forecast for Los Angeles, CA in Fahrenheit:
Day 1: Max Temp: 71.0° Fahrenheit, Min Temp: 60.1° Fahrenheit
Day 2: Max Temp: 78.5° Fahrenheit, Min Temp: 57.0° Fahrenheit
Day 3: Max Temp: 84.9° Fahrenheit, Min Temp: 55.9° Fahrenheit



In [26]:
get_n_day_weather_forecast('Los Angeles, CA', 'fahrenheit',  3)

'3-day weather forecast for Los Angeles, CA in Fahrenheit:\nDay 1: Max Temp: 71.0° Fahrenheit, Min Temp: 60.1° Fahrenheit\nDay 2: Max Temp: 78.5° Fahrenheit, Min Temp: 57.0° Fahrenheit\nDay 3: Max Temp: 84.9° Fahrenheit, Min Temp: 55.9° Fahrenheit\n'

In [27]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Los Angeles, CA over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content='Could you please specify the number of days ("x days") you would like the weather forecast for Los Angeles, CA? Additionally, let me know if you prefer the temperature in Celsius or Fahrenheit.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [28]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]



Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Would you like the temperature in Celsius or Fahrenheit for the 5-day weather forecast in Los Angeles, CA?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))

In [29]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_O29aLpI4DXmRtYEqjQ5kY18b', function=Function(arguments='{"location":"Toronto, Canada","format":"celsius","num_days":1}', name='get_n_day_weather_forecast'), type='function')])

In [30]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0].message

ChatCompletionMessage(content='Would you like the current weather in Toronto, or a weather forecast for the upcoming days? Additionally, do you prefer the temperature in Celsius or Fahrenheit?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [31]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message


ChatCompletionMessage(content='Sure, I will get the current weather in Celsius for Toronto, Canada. Please hold on a moment.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [32]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

[ChatCompletionMessageToolCall(id='call_nCS3uEkFyST9TNPQoAxSqZ4h', function=Function(arguments='{"location": "San Francisco, CA", "format": "fahrenheit", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'),
 ChatCompletionMessageToolCall(id='call_rMtF8O4nGpKWdQxLVEsib7IC', function=Function(arguments='{"location": "Glasgow, UK", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]